`INFO 612 - LAB 2`

### SPATIAL STATISTICS
---

For this lab I'm using the ArcGIS project previously created for lab 1 where I mapped all the Chinese restaurants across New York State and the NYC metropolitan area, according to the Foursquare Open Source Places dataset.  In this map I've separated restaurants into two point layers, urban and rural locations, according to the 2020 US Census Urban Areas TIGER/Line shapefile.  

This time I'm using the NCES Locales instead for more granular urban/rural classifications, and I will bring in all the restaurant points for the entire US.  

In [1]:
import arcpy
from arcgis import GIS
import csv

In [2]:
# define paths to the multiple gdbs
main_gdb = r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\chinese-restaurants.gdb"
nces_gdb = r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\NCES_locales\5cfc2043-145e-48f7-a356-66c8e138838b.gdb"

# toggle on: overwriting existing feature classes
arcpy.env.overwriteOutput = True

In [3]:
# set workspace path
arcpy.env.workspace = main_gdb

In [4]:
# use current project
aprx = arcpy.mp.ArcGISProject("CURRENT")

# access the existing map object (index 0 because I only have one)
map_obj = aprx.listMaps("Map")[0]

First, I'll need to set the workspace to the NCES geodatabase to get the locales layer into the project, so I'll temporarily set the workspace to `nces_gdb`

In [29]:
# set workspace to NCES to get the locales layer
arcpy.env.workspace = nces_gdb

In [33]:
# list feature classes of current gdb
arcpy.ListFeatureClasses()

['EDGE_LOCALE_CURRENT']

In [32]:
# add locales to map in main geodatabase
arcpy.management.MakeFeatureLayer('EDGE_LOCALE_CURRENT')

<Result 'EDGE_LOCALE_CURRENT_Layer1'>

In [34]:
# set workspace back to main
arcpy.env.workspace = main_gdb

In [35]:
# list layer names
for layer in map_obj.listLayers():
    print(layer.name)

Light Gray Reference
EDGE_LOCALE_CURRENT_Layer
NY-NJ-CT-PA
tristate_restaurants_rural
tristate_urban
tristate_restaurants_urban
tl_2020_us_state_tristate
tl_2020_us_state
tl 2020 census urban
Light Gray Base


In [36]:
# rename the NCES layer
for layer in map_obj.listLayers():
    if "EDGE_LOCALE_CURRENT_Layer" in layer.name:
        layer.name = "NCES_Locales"

# list layers again to check that it was renamed
for layer in map_obj.listLayers():
    print(layer.name)

Light Gray Reference
NCES_Locales
NY-NJ-CT-PA
tristate_restaurants_rural
tristate_urban
tristate_restaurants_urban
tl_2020_us_state_tristate
tl_2020_us_state
tl 2020 census urban
Light Gray Base


### [NCES Classifications 2024](https://nces.ed.gov/programs/edge/Geographic/LocaleBoundaries)
---

**CITIES** - inside an urban area with population 50,000 or more, inside a principal city

`11` Large City - 250,000+

`12` Midsize City - 100,000 - 250,000

`13` Small City - < 100,000

**SUBURBAN** - inside an urban area, outside a principal city

`21` Large Suburban - 250,000+

`22` Midsize Suburban - 100,000 - 250,000

`23` Small Suburban - 50,000 - 100,000

**TOWN** - inside an urban area with population less than 50,000

`31` Fringe Town - less than 10 miles from an urban area with 50,000+

`32` Distant Town - 10-35 miles from an urban area with 50,000+

`33` Remote Town - 35+ miles from an urban area with 50,000+

**RURAL** - outside an urban area

`41` Fringe Rural - < 5 miles from an urban area of 50,000+ or < 2.5 miles from urban area with < 50,000

`42` Distant Rural - 5-25 miles from an urban area of 50,000+ or > 2.5 miles from urban area with < 50,000

`43` Remote Rural - > 25 miles from an urban area of 50,000+ and > 10 miles from urban area with < 50,000

In [43]:
# bring in the locale classifications to join with the attribute table to make the table more readable
csv_path = "NCES_locales.csv"

# create a dictionary from the csv
locale_lookup = {}
with open(csv_path, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        locale_lookup[row['LOCALE']] = row['LOC_NAME']

In [44]:
# add new field to the attribute table for the NCES_Locales layer
arcpy.management.AddField(
    in_table="NCES_Locales",
    field_name="LOC_NAME",
    field_type="TEXT",
    field_alias="Locale Name",
)

<Result 'NCES_Locales'>

In [50]:
# fill in locale names based on locale codes (thanks, genAI)
# also, make sure nothing is selected otherwise this won't work
with arcpy.da.UpdateCursor("NCES_Locales", ['LOCALE', 'LOC_NAME']) as cursor:
    for row in cursor:
        locale_code = str(row[0]) if row[0] is not None else None
        if locale_code and locale_code in locale_lookup:
            row[1] = locale_lookup[locale_code]
            cursor.updateRow(row)


In [52]:
# create a separate feature layer for each of the 12 locales (thanks, genAI)
for locale_code, loc_name in locale_lookup.items():
    # SQL where clause to filter by locale code
    where_clause = f"LOCALE = '{locale_code}'"
    
    # create layer name
    layer_name = f"{loc_name} ({locale_code})"
    
    # create feature layer named after the locale name with code
    arcpy.management.MakeFeatureLayer(
        in_features="NCES_Locales",
        out_layer=layer_name,
        where_clause=where_clause
    )
    print(f"Created feature layer: {layer_name}")

Created feature layer: Large City (11)
Created feature layer: Midsize City (12)
Created feature layer: Small City (13)
Created feature layer: Large Suburban (21)
Created feature layer: Midsize Suburban (22)
Created feature layer: Small Suburban (23)
Created feature layer: Fringe Town (31)
Created feature layer: Distant Town (32)
Created feature layer: Remote Town (33)
Created feature layer: Fringe Rural (41)
Created feature layer: Distant Rural (42)
Created feature layer: Remote Rural (43)


### Applying Symbology
---
Now I can already see quite a difference between the census and NCES classifications.  For example, most of Long Island was classified as urban according to the census, but is a "large suburban" area according to NCES.  

Next, I manually updated the symbology of these new layers to group the territories by colour (city, suburban, town, and rural).  After this I realized I could also apply a continuous colour scheme over the locale names of the single NCES_Locales layer.  

### Bringing in the restaurants
---
The CSV of Chinese restaurants that I created from Foursquare's Open Space Places data includes all the restaurants in Canada and the US.  I organized the restaurants into two layers, one for each country, and applied the same pink symbology to the new layers.  

In [56]:
# create new feature layer from csv
arcpy.management.XYTableToPoint(
    in_table=r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\chinese_restaurants_cleaned.csv",
    out_feature_class=r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\chinese-restaurants.gdb\usa_restaurants",
    x_field="longitude",
    y_field="latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'C:\\Users\\jocel\\Documents\\ArcGIS\\Projects\\chinese-restaurants\\chinese-restaurants.gdb\\usa_restaurants'>

Since the Foursquare data uses WGS84 coordinates, I'll convert the data permanently so it's not projecting on-the-fly. 

In [57]:
# change restaurants feature layer coordinates to match the map's projection permanently
arcpy.management.Project(
    in_dataset="usa_restaurants",
    out_dataset=r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\chinese-restaurants.gdb\usa_restaurants_albers",
    out_coor_system='PROJCS["North_America_Albers_Equal_Area_Conic",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",20.0],PARAMETER["Standard_Parallel_2",60.0],PARAMETER["Latitude_Of_Origin",40.0],UNIT["Meter",1.0]]',
    transform_method="WGS_1984_(ITRF00)_To_NAD_1983",
    in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    preserve_shape="NO_PRESERVE_SHAPE",
    max_deviation=None,
    vertical="NO_VERTICAL"
)

<Result 'C:\\Users\\jocel\\Documents\\ArcGIS\\Projects\\chinese-restaurants\\chinese-restaurants.gdb\\usa_restaurants_albers'>

In [9]:
# checking the layer names again to verify which one i want to remove
am = aprx.activeMap
for layer in am.listLayers():
    print(layer.name)

Dark Gray Reference
canada_restaurants
us_restaurants
tl_2020_us_state
us_restaurants_firefly
RURAL
Remote Rural (43)
Distant Rural (42)
Fringe Rural (41)
TOWNS
Remote Town (33)
Distant Town (32)
Fringe Town (31)
SUBURBS
Small Suburban (23)
Midsize Suburban (22)
Large Suburban (21)
CITIES
Small City (13)
Midsize City (12)
Large City (11)
NCES_Locales
NY-NJ-CT-PA
tristate_restaurants_rural
tristate_urban
tristate_restaurants_urban
tl_2020_us_state_tristate
tl 2020 census urban
Dark Gray Base


In [64]:
# remove the original layer
layer_to_remove = map_obj.listLayers("usa_restaurants")[0]
map_obj.removeLayer(layer_to_remove)

In [74]:
# rename the reprojected layer
for layer in map_obj.listLayers():
    if "usa_restaurants_albers" in layer.name:
        layer.name = "us_restaurants"

In [72]:
# make a new layer for canadian restaurants only
arcpy.management.MakeFeatureLayer(
    in_features="us_restaurants",
    out_layer="canada_restaurants",
    where_clause="country = 'CA'",
    workspace=None,
    field_info="OBJECTID OBJECTID VISIBLE NONE;Shape Shape VISIBLE NONE;fsq_place_id fsq_place_id VISIBLE NONE;name name VISIBLE NONE;latitude latitude VISIBLE NONE;longitude longitude VISIBLE NONE;address address VISIBLE NONE;locality locality VISIBLE NONE;region region VISIBLE NONE;postcode postcode VISIBLE NONE;admin_region admin_region VISIBLE NONE;post_town post_town VISIBLE NONE;po_box po_box VISIBLE NONE;country country VISIBLE NONE;date_created date_created VISIBLE NONE;date_refreshed date_refreshed VISIBLE NONE;date_closed date_closed VISIBLE NONE;tel tel VISIBLE NONE;website website VISIBLE NONE;email email VISIBLE NONE;facebook_id facebook_id VISIBLE NONE;instagram instagram VISIBLE NONE;twitter twitter VISIBLE NONE;fsq_category_ids fsq_category_ids VISIBLE NONE;fsq_category_labels fsq_category_labels VISIBLE NONE;placemaker_url placemaker_url VISIBLE NONE;unresolved_flags unresolved_flags VISIBLE NONE;geom geom VISIBLE NONE;bbox bbox VISIBLE NONE"
)

<Result 'canada_restaurants'>

In [67]:
# delete canadian restaurants from us layer
# first make the selection
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="us_restaurants",
    selection_type="NEW_SELECTION",
    where_clause="country = 'CA'",
    invert_where_clause=None
)

<Result 'us_restaurants'>

In [ ]:
# !!! this deleted too much, see note below
# make sure selection is active before running
# delete canadian restaurants
arcpy.management.DeleteFeatures("us_restaurants")

Whoops. That removed the canadian restaurants from all the layers that were made from usa_restaurants_albers in the gdb. I mistakenly thought the feature class in the gdb wouldn't be affected.  I'll have to import canadian restarants again.

In [75]:
# apply the same pink symbology as the tristate restaurants
arcpy.management.ApplySymbologyFromLayer(
    in_layer="us_restaurants",
    in_symbology_layer=r"NY-NJ-CT-PA\tristate_restaurants_rural",
    symbology_fields=None,
    update_symbology="DEFAULT"
)

<Result 'us_restaurants'>

In [76]:
# attempting to bring back canadian restaurants
arcpy.management.Project(
    in_dataset="usa_restaurants",
    out_dataset=r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\chinese-restaurants.gdb\canada_restaurants",
    out_coor_system='PROJCS["North_America_Albers_Equal_Area_Conic",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",20.0],PARAMETER["Standard_Parallel_2",60.0],PARAMETER["Latitude_Of_Origin",40.0],UNIT["Meter",1.0]]',
    transform_method="WGS_1984_(ITRF00)_To_NAD_1983",
    in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    preserve_shape="NO_PRESERVE_SHAPE",
    max_deviation=None,
    vertical="NO_VERTICAL"
)

<Result 'C:\\Users\\jocel\\Documents\\ArcGIS\\Projects\\chinese-restaurants\\chinese-restaurants.gdb\\canada_restaurants'>

In [78]:
# delete us restaurants from canada layer
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="canada_restaurants",
    selection_type="NEW_SELECTION",
    where_clause="country = 'US'",
    invert_where_clause=None
)

<Result 'canada_restaurants'>

In [79]:
# make sure selection is active
arcpy.management.DeleteFeatures("canada_restaurants")

<Result 'canada_restaurants'>

In [80]:
# apply the same pink symbology as the us restaurants
arcpy.management.ApplySymbologyFromLayer(
    in_layer="canada_restaurants",
    in_symbology_layer=r"us_restaurants",
    symbology_fields=None,
    update_symbology="DEFAULT"
)

<Result 'canada_restaurants'>

**Note**: After applying the symbology, I manually changed the Feature Blend mode of these layers to 'Linear Dodge' so that they melt together when overlapping.  

### Bringing in ACS data on race and ethnicity
---
I used `R` to gather census data on race and ethnicity by county and census tract.  To do this I had to create a variable with the shortcut `state.abb` to include all the states plus DC and Puerto Rico, since the Foursquare data for the US includes PR:

`all_states <- c(state.abb, "DC", "PR")`

Then I used `get_acs()` for the 2023 5-Year estimates.  This wasn't easily found in the tidycensus documentation, but was successfully revealed by generative AI. 

In [6]:
# add the race and ethnicity shapefiles exported from R

# by census tract
path_race_tract = r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\census\acs2023_race_by_tract.shp"
path_ethnicity_tract = r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\census\acs2023_ethnicity_by_tract.shp"

# by county
path_race_county = r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\census\acs2023_race_by_county.shp"
path_ethnicity_county = r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\census\acs2023_ethnicity_by_county.shp"

map_obj.addDataFromPath(path_race_tract)
map_obj.addDataFromPath(path_ethnicity_tract)
map_obj.addDataFromPath(path_race_county)
map_obj.addDataFromPath(path_ethnicity_county)

I manually applied unclassed colour symbology using the Inferno scheme to highlight all the census tracts that are at least 30% Chinese.  

There are no census tracts that have 100% Chinese residents, the most is 88%.  Overwhelmingly, most census tracts have 10% or fewer Chinese people.  

In [7]:
# copy symbology to race layer, I'd like to see how the same illustrates white populations
arcpy.management.ApplySymbologyFromLayer(
    in_layer="acs2023_race_by_tract",
    in_symbology_layer="acs2023_ethnicity_by_tract",
    symbology_fields="NORMALIZATION_FIELD chinese whiteE",
    update_symbology="MAINTAIN"
)

<Result 'acs2023_race_by_tract'>

Unlike Chinese populations, there are a lot of census tracts that consist of 100% white people.  At a glance, it looks like these (along with tracts that are at least 30% white) tend to be more rural, and places where there are Chinese communities tend to be in cities.  

### Calculating the relationship between locale type and Chinese population clusters
---
To get a quantitative measure of this observation I'll calculate ################################################.

In [10]:
# calculating the percentage of chinese people into a new field
arcpy.management.CalculateField(
    in_table="acs2023_ethnicity_by_tract_Ex",
    field="ch_pct",
    expression="0 if !pop! == 0 else ((!chinese! * 100) / !pop!)",
    expression_type="PYTHON3",
    code_block="",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'acs2023_ethnicity_by_tract_Ex'>

In [1]:
# hot spot analysis on chinese population
arcpy.stats.HotSpots(
    Input_Feature_Class="acs2023_ethnicity_by_tract_Ex",
    Input_Field="ch_pct",
    Output_Feature_Class=r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\chinese-restaurants.gdb\chinese_HotSpots",
    Conceptualization_of_Spatial_Relationships="FIXED_DISTANCE_BAND",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Self_Potential_Field=None,
    Weights_Matrix_File=None,
    Apply_False_Discovery_Rate__FDR__Correction="APPLY_FDR",
    number_of_neighbors=None
)

<Result 'C:\\Users\\jocel\\Documents\\ArcGIS\\Projects\\chinese-restaurants\\chinese-restaurants.gdb\\chinese_HotSpots'>

In [2]:
# run summarize within to see which locale types are associated with higher or lower percentages of Chinese people
arcpy.analysis.SummarizeWithin(
    in_polygons="NCES_Locales",
    in_sum_features="acs2023_ethnicity_by_tract_Ex",
    out_feature_class=r"C:\Users\jocel\Documents\ArcGIS\Projects\chinese-restaurants\chinese-restaurants.gdb\acs2023_ethnicity_by_tract_Ex_SummarizeWithin",
    keep_all_polygons="KEEP_ALL",
    sum_fields="ch_pct Mean",
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)

<Result 'C:\\Users\\jocel\\Documents\\ArcGIS\\Projects\\chinese-restaurants\\chinese-restaurants.gdb\\acs2023_ethnicity_by_tract_Ex_SummarizeWithin'>

In [3]:
# running spatial autocorrelation with the results of Summarize Within (thanks to generative AI for the idea to combine these)
arcpy.stats.SpatialAutocorrelation(
    Input_Feature_Class="acs2023_ethnicity_by_tract_Ex_SummarizeWithin",
    Input_Field="mean_ch_pct",
    Generate_Report="GENERATE_REPORT",
    Conceptualization_of_Spatial_Relationships="CONTIGUITY_EDGES_ONLY",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Weights_Matrix_File=None,
    number_of_neighbors=None
)

<Result '0.435678'>

### Next steps
---

The p-value shows that there is a statistically significant spatial clustering that mostly follows the urban-rural gradient.  Chinese populations prefer to cluster in cities and suburban areas, which was already visible when I mapped the percentage of Chinese people per census tract.  

Next I'd like to run similar analyses on the Chinese restaurant locations.  I did some quick analysis by state which was interesting:  the two states with the most Chinese residents had the fewest Chinese restaurants per 100 Chinese people.  Maybe this is because places like New York and San Francisco are densely populated, shorter distances to the nearest Chinese restaurant may negate the need for more restaurants.  

Visually, a map of Chinese restaurant locations across the US does show dense clusters in those cities, but they also appear quite evenly distributed across the country, with at least one restaurant even in the most remote locales.  I'm curious if Chinese restaurants locations exhibit a spatial structure that follows the urban-rural gradient, of if it's different somehow.  